<a href="https://colab.research.google.com/github/vishwasjoshi2019/Stock_Market_Analysis/blob/main/TAKESHI_STRATEGY_STOCKS(filtering_stocks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29089 sha256=8dfbdd200d3ba0aee052b57c3463218677488b81ef10aa810b23e4234077fffb
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
import ta



In [ ]:

# Step 1: Fetch a list of S&P 500 stocks
sp500 = yf.download('^GSPC', period='1d', progress=False)


In [ ]:
file_path = 'S&P500.xlsx'

# Read the Excel file and extract the first column as the index
sp500_components_df = pd.read_excel(file_path, index_col=0)

# Get the list of S&P 500 components from the index column
sp500_tickers = sp500_components_df.index.tolist()

In [ ]:
# Define a function to calculate RSI
def calculate_rsi(df, window=14):
    # Create an RSIIndicator instance
    rsi = ta.momentum.RSIIndicator(close=df['Close'], window=window)

    # Add RSI values to the DataFrame
    df['rsi'] = rsi.rsi()

    return df

# Define a function to calculate MACD
def calculate_macd(df, window_slow=26, window_fast=12, window_sign=9):
    # Create a MACDIndicator instance
    macd = ta.trend.MACD(close=df['Close'], window_slow=window_slow, window_fast=window_fast, window_sign=window_sign)

    # Add MACD values to the DataFrame
    df['macd'] = macd.macd()
    df['signal_line'] = macd.macd_signal()

    return df


# Define a function to calculate moving averages
def calculate_moving_averages(df, moving_average_periods=[7, 10, 15, 20, 25]):
    # Calculate moving averages for the specified periods
    for period in moving_average_periods:
        df[f'SMA_{period}'] = df['Close'].rolling(window=period).mean()

    return df


In [ ]:

# Initialize an empty dictionary to store data for each stock
stock_data_dict = {}

# Loop through each stock
for ticker in sp500_tickers:
    try:
        # Fetch historical data for the stock for the past year
        stock_data = yf.download(ticker, period='1y', progress=False)

        # Calculate RSI using the function
        stock_data = calculate_rsi(stock_data)

        # Calculate MACD using the function
        stock_data = calculate_macd(stock_data)

        # Calculate moving averages using the function
        stock_data = calculate_moving_averages(stock_data)

        # Store the DataFrame with RSI, MACD, and moving averages in the dictionary
        stock_data_dict[ticker] = stock_data

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")



In [ ]:
stock_data_dict['AAPL']

,Open,High,Low,Close,Adj Close,Volume,rsi,macd,signal_line,SMA_7,SMA_10,SMA_15,SMA_20,SMA_25
Date,,,,,,,,,,,,,,
2022-10-07,142.539993,143.100006,139.449997,140.089996,139.263840,85925600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-10,140.419998,141.889999,138.570007,140.419998,139.591888,74899000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-11,139.899994,141.350006,138.220001,138.979996,138.160370,77033700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-12,139.130005,140.360001,138.160004,138.339996,137.524139,70433700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-13,134.990005,143.589996,134.369995,142.990005,142.146729,113224000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-02,171.220001,174.300003,170.929993,173.750000,173.750000,52164500,44.034205,-2.599168,-2.335791,172.701429,173.740001,174.442001,176.217001,178.145200
2023-10-03,172.259995,173.630005,170.820007,172.399994,172.399994,49594600,41.791997,-2.534517,-2.375536,172.360001,173.073000,174.182001,175.352000,177.833600
2023-10-04,171.089996,174.210007,170.970001,173.660004,173.660004,53020300,44.626088,-2.354468,-2.371322,172.014287,172.889999,174.145334,174.889500,177.415200


In [52]:
# Define a function to filter stocks based on the conditions
def filter_stocks(stock_data_dict, ma_period, rsi_lower_limit=35, rsi_upper_limit=40):
    filtered_stocks = []

    for ticker, stock_data in stock_data_dict.items():
        try:
            # Get the current price
            current_price = stock_data['Close'][-1]

            # Get the specified moving average for the stock
            ma_column = f'SMA_{ma_period}'
            ma_value = stock_data[ma_column][-1]

            # Check if the current price is less than or equal to 85% of the moving average
            price_condition = current_price <= 0.95 * ma_value

            # Calculate RSI for the stock
            rsi = stock_data['rsi'][-1]

            # Check if RSI is within the specified range (rsi_lower_limit <= RSI <= rsi_upper_limit)
            rsi_condition = rsi >= rsi_lower_limit and rsi <= rsi_upper_limit

            # If both conditions are met, add the ticker to the filtered list
            if price_condition and rsi_condition:
                filtered_stocks.append(ticker)

        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")

    return filtered_stocks

# Define the moving average periods to check (7, 10, 15, 20, 25)
moving_average_periods_to_check = [7, 10, 15, 20, 25]

# Initialize an empty dictionary to store filtered stocks for each moving average period
filtered_stocks_dict = {}

# Loop through the specified moving average periods and filter stocks
for ma_period in moving_average_periods_to_check:
    filtered_stocks = filter_stocks(stock_data_dict, ma_period)
    filtered_stocks_dict[f'MA_{ma_period}'] = filtered_stocks

# Print the filtered stocks for each moving average period
for ma_period, filtered_stocks in filtered_stocks_dict.items():
    print(f"Stocks with current price <= 85% of {ma_period}-day MA, RSI between 20 and 40:")
    print(filtered_stocks)


Stocks with current price <= 85% of MA_7-day MA, RSI between 20 and 40:
[]
Stocks with current price <= 85% of MA_10-day MA, RSI between 20 and 40:
['MRO']
Stocks with current price <= 85% of MA_15-day MA, RSI between 20 and 40:
['SLB', 'MRO', 'LNC']
Stocks with current price <= 85% of MA_20-day MA, RSI between 20 and 40:
['SLB', 'MPC', 'GM', 'BKR', 'MRO', 'NI', 'KEY', 'BBWI', 'CTLT', 'VFC', 'LNC']
Stocks with current price <= 85% of MA_25-day MA, RSI between 20 and 40:
['SLB', 'EOG', 'MPC', 'GM', 'DLR', 'BKR', 'GPN', 'PHM', 'IRM', 'MRO', 'NI', 'KEY', 'BBWI', 'CTLT', 'VFC', 'LNC']


Brown-Forman Corporation (Ticker: BF-B) data not found in the dictionary.
